# [ Lab 9-4 : XOR ]


In [1]:
# Lab 9 XOR
import os
# silence INFO logs set it to 1
# filter out WARNING set it to 2 
# silence ERROR logs (not recommended) set it to 3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np


print('tensorflow version: {0}'.format(tf.__version__))
print('numpy version: {0}'.format(np.__version__))


tensorflow version: 1.2.0
numpy version: 1.13.0


# For reproducibility

In [2]:
tf.set_random_seed(777)  # for reproducibility


# Init. variables

In [3]:
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')



# Layer1

In [4]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)




# Layer2

In [5]:
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)



# Cost/loss function

In [6]:
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)



# Training

In [7]:
with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



# Accuracy computation
>  True if hypothesis > 0.5 else False

In [8]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)



# Launch graph

In [9]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


'''
Hypothesis:  [[ 0.01338218]
 [ 0.98166394]
 [ 0.98809403]
 [ 0.01135799]]
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy:  1.0
'''

0 0.715638 [array([[ 0.79269594,  0.68861032],
       [-1.20728338, -0.29517072]], dtype=float32), array([[ 1.7177    ],
       [ 0.35572484]], dtype=float32)]
100 0.690733 [array([[ 0.32268241, -0.23303069],
       [-0.77434164,  0.80529422]], dtype=float32), array([[ 1.46081126],
       [ 0.78415203]], dtype=float32)]
200 0.495032 [array([[ 1.67216921, -2.3216548 ],
       [-2.21686363,  2.656147  ]], dtype=float32), array([[ 2.82439113],
       [ 2.60258126]], dtype=float32)]
300 0.223746 [array([[ 3.18776298, -3.74428821],
       [-3.69724321,  4.12956667]], dtype=float32), array([[ 4.70040798],
       [ 4.31880617]], dtype=float32)]
400 0.122295 [array([[ 3.98867273, -4.52083349],
       [-4.4841404 ,  4.90343523]], dtype=float32), array([[ 5.8487258],
       [ 5.4189949]], dtype=float32)]
500 0.0788432 [array([[ 4.48614645, -5.01518679],
       [-4.97325087,  5.39076948]], dtype=float32), array([[ 6.64474821],
       [ 6.20202589]], dtype=float32)]
600 0.0559812 [array([[ 4.83938

5000 0.000789405 [array([[ 7.84969997, -8.45132446],
       [-8.30820274,  8.76673508]], dtype=float32), array([[ 15.14112568],
       [ 14.76316071]], dtype=float32)]
5100 0.000746024 [array([[ 7.8787756 , -8.48109531],
       [-8.33723164,  8.79612732]], dtype=float32), array([[ 15.24907303],
       [ 14.87186241]], dtype=float32)]
5200 0.000705329 [array([[ 7.9074769 , -8.51048279],
       [-8.36588955,  8.82514381]], dtype=float32), array([[ 15.35638809],
       [ 14.97991657]], dtype=float32)]
5300 0.000667051 [array([[ 7.9358201 , -8.53950214],
       [-8.39419079,  8.85379791]], dtype=float32), array([[ 15.46310806],
       [ 15.08736515]], dtype=float32)]
5400 0.000631071 [array([[ 7.96382141, -8.56816483],
       [-8.42215824,  8.88210392]], dtype=float32), array([[ 15.56927395],
       [ 15.19424057]], dtype=float32)]
5500 0.000597195 [array([[ 7.99149418, -8.59649181],
       [-8.44979286,  8.91008186]], dtype=float32), array([[ 15.67490673],
       [ 15.30058002]], dtype=fl

9900 6.13649e-05 [array([[ 9.00449181, -9.63154984],
       [-9.46273327,  9.93393517]], dtype=float32), array([[ 20.07340431],
       [ 19.72282982]], dtype=float32)]
10000 5.83398e-05 [array([[ 9.02439404, -9.65184593],
       [-9.48265266,  9.95403957]], dtype=float32), array([[ 20.17106056],
       [ 19.82086754]], dtype=float32)]

Hypothesis:  [[  6.13103257e-05]
 [  9.99936938e-01]
 [  9.99950767e-01]
 [  5.97514700e-05]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0


'\nHypothesis:  [[ 0.01338218]\n [ 0.98166394]\n [ 0.98809403]\n [ 0.01135799]]\nCorrect:  [[ 0.]\n [ 1.]\n [ 1.]\n [ 0.]]\nAccuracy:  1.0\n'